# Milestone 4: RAG (Retrieval Augmented Generation) with Google Gemini

## Overview

This notebook demonstrates the **RAG pipeline** for answering natural language queries about financial data using:

- **Memory Service**: Stores user profile, budgets, and conversation history
- **LLM Service**: Google Gemini for natural language generation
- **RAG Insights Tool**: Combines retrieval + context + LLM

## RAG Workflow:

1. **Retrieval**: Load relevant transactions from CSV
2. **Context Compaction**: Filter and summarize transaction data
3. **Prompt Construction**: Combine query + memory + transactions
4. **LLM Call**: Generate natural language response
5. **Return**: User-friendly answer

---

**Author**: Alfred Munga  
**Date**: November 18, 2025  
**Milestone**: 4 - RAG Implementation

## 1. Setup and Imports

In [ ]:
import os
import sys
from pathlib import Path
import pandas as pd
from dotenv import load_dotenv

# Add project root to path
project_root = Path.cwd().parent
sys.path.append(str(project_root))

# Load environment variables
load_dotenv(project_root / '.env')

print("✅ Environment loaded")
print(f"📁 Project Root: {project_root}")

## 2. Check API Key

⚠️ **Important**: You need a Google Gemini API key to run this notebook.

Get your free API key at: https://makersuite.google.com/app/apikey

Then add it to your `.env` file:
```
GEMINI_API_KEY=your-api-key-here
```

In [ ]:
# Check if API key is available
api_key = os.getenv('GEMINI_API_KEY')

if api_key:
    print("✅ GEMINI_API_KEY found")
    print(f"   Key preview: {api_key[:10]}...{api_key[-4:]}")
else:
    print("❌ GEMINI_API_KEY not found!")
    print("\nTo use this notebook:")
    print("1. Get a free API key: https://makersuite.google.com/app/apikey")
    print("2. Add to .env file: GEMINI_API_KEY=your-key-here")
    print("3. Restart this notebook")

## 3. Initialize Memory Bank

The **MemoryBank** stores:
- User profile (name, business type)
- Budgets by category
- Conversation history (last 5 interactions)

In [ ]:
from agent.memory import MemoryBank, UserProfile

# Create user profile
profile = UserProfile(
    name="Jane Wanjiru",
    business_type="Retail Shop",
    business_name="Wanjiru's Convenience Store",
    location="Nairobi, Kenya"
)

# Define budgets (KES per month)
budgets = {
    "transport": 5000,
    "food": 3000,
    "utilities": 4000,
    "supplies": 10000
}

# Initialize memory bank
memory = MemoryBank(user_profile=profile, budgets=budgets)

print("✅ Memory Bank initialized")
print("\n📊 Summary:")
summary = memory.get_summary()
for key, value in summary.items():
    print(f"   {key}: {value}")

### View Memory Context

In [ ]:
# Get formatted context
context = memory.get_context()

print("📋 Memory Context:")
print("=" * 60)
print(context)
print("=" * 60)

## 4. Load Transaction Data

Load the synthetic SMS transaction data generated in Milestone 2.

In [ ]:
# Load transactions
data_path = project_root / "data" / "synthetic" / "sms.csv"
transactions_df = pd.read_csv(data_path)
transactions_df['date'] = pd.to_datetime(transactions_df['date'])

print(f"✅ Loaded {len(transactions_df)} transactions")
print(f"\n📊 Transaction Types:")
print(transactions_df['transaction_type'].value_counts())

# Show sample
print("\n🔍 Sample Transactions:")
transactions_df[['date', 'transaction_type', 'amount', 'sender_recipient']].head()

## 5. Initialize RAG Insights Tool

The **RAGInsightsTool** combines:
- Transaction data retrieval
- Context management
- LLM-powered responses

In [ ]:
from tools.rag_insights_tool import RAGInsightsTool

# Initialize RAG tool
print("🔧 Initializing RAG Insights Tool...")

try:
    rag_tool = RAGInsightsTool(
        memory=memory,
        data_path=str(data_path)
    )
    print("✅ RAG Tool initialized successfully!")
    
    # Get transaction summary
    summary = rag_tool.get_transaction_summary()
    print("\n📊 Transaction Summary:")
    print(f"   Total Transactions: {summary['total_transactions']}")
    print(f"   Total Spent: KES {summary['total_spent']:,.2f}")
    print(f"   Total Received: KES {summary['total_received']:,.2f}")
    print(f"   Net Flow: KES {summary['net_flow']:,.2f}")
    
except ValueError as e:
    print(f"❌ Error: {str(e)}")
    print("\n⚠️  Make sure GEMINI_API_KEY is set in your .env file")

## 6. Demo Queries

Let's ask the RAG system some natural language questions!

### Query 1: Transport Spending

In [ ]:
query1 = "How much have I spent on transport this month?"

print(f"❓ Query: {query1}")
print("\n🤔 Processing...\n")

response1 = rag_tool.run(query1)

print("💬 Response:")
print("=" * 60)
print(response1)
print("=" * 60)

### Query 2: Budget Check

In [ ]:
query2 = "Am I exceeding my food budget?"

print(f"❓ Query: {query2}")
print("\n🤔 Processing...\n")

response2 = rag_tool.run(query2)

print("💬 Response:")
print("=" * 60)
print(response2)
print("=" * 60)

### Query 3: M-Pesa Summary

In [ ]:
query3 = "Summarize my recent M-Pesa transactions"

print(f"❓ Query: {query3}")
print("\n🤔 Processing...\n")

response3 = rag_tool.run(query3)

print("💬 Response:")
print("=" * 60)
print(response3)
print("=" * 60)

### Query 4: Financial Advice

In [ ]:
query4 = "What advice do you have for improving my cash flow?"

print(f"❓ Query: {query4}")
print("\n🤔 Processing...\n")

response4 = rag_tool.run(query4)

print("💬 Response:")
print("=" * 60)
print(response4)
print("=" * 60)

## 7. View Conversation History

The memory bank stores the last 5 conversations.

In [ ]:
print("📜 Conversation History:")
print("=" * 60)

for i, entry in enumerate(memory.conversation_history, 1):
    print(f"\n{i}. [{entry.timestamp}]")
    print(f"   User: {entry.user_query}")
    print(f"   Assistant: {entry.assistant_response[:100]}...")
    if entry.context_used:
        print(f"   Context: {entry.context_used}")

print("\n" + "=" * 60)

## 8. RAG Architecture Visualization

Here's how the RAG pipeline works:

```
┌─────────────────────────────────────────────────────────┐
│                    User Query                           │
│          "How much have I spent on transport?"          │
└──────────────────────┬──────────────────────────────────┘
                       │
                       ▼
┌─────────────────────────────────────────────────────────┐
│              1. RETRIEVAL                               │
│   Filter transactions by keywords (transport, fuel)     │
│   → Returns 20 most relevant transactions              │
└──────────────────────┬──────────────────────────────────┘
                       │
                       ▼
┌─────────────────────────────────────────────────────────┐
│         2. CONTEXT COMPACTION                           │
│   Summarize transactions into readable format           │
│   → "Total: KES 3,500 (Shell, Uber, Bolt)"             │
└──────────────────────┬──────────────────────────────────┘
                       │
                       ▼
┌─────────────────────────────────────────────────────────┐
│         3. PROMPT CONSTRUCTION                          │
│   Combine: User Profile + Budgets + Transactions       │
│   → Full context for LLM                               │
└──────────────────────┬──────────────────────────────────┘
                       │
                       ▼
┌─────────────────────────────────────────────────────────┐
│              4. LLM CALL                                │
│   Google Gemini generates natural language response    │
│   → "You've spent KES 3,500 on transport..."           │
└──────────────────────┬──────────────────────────────────┘
                       │
                       ▼
┌─────────────────────────────────────────────────────────┐
│         5. UPDATE MEMORY                                │
│   Store query + response in conversation history       │
│   → Available for future context                       │
└─────────────────────────────────────────────────────────┘
```

## 9. Key ADK Concepts Demonstrated

### 1. **Context Engineering**
- User profile provides personalization
- Budget data enables comparison
- Transaction history gives factual grounding
- Conversation history maintains continuity

### 2. **Memory Management**
- Sliding window (last 5 conversations)
- Persistent user preferences
- Structured data (profiles, budgets)
- Efficient context retrieval

### 3. **RAG Pipeline**
- Smart retrieval (keyword-based filtering)
- Context compaction (summarization)
- Prompt engineering (system + context + query)
- Natural language generation

### 4. **Tool Design**
- Clear separation of concerns
- Reusable components
- Error handling
- Extensibility

## 10. Summary

✅ **Completed:**
- LLM Service with Google Gemini
- Memory Bank for context management
- RAG Insights Tool
- Natural language query interface

🎯 **Key Achievements:**
- Demonstrated full RAG pipeline
- Integrated memory with LLM
- Answered financial queries in natural language
- Maintained conversation context

🚀 **Next Steps:**
- Add more sophisticated retrieval (semantic search)
- Implement function calling for actions
- Add visualization generation
- Deploy as API endpoints